# **Mini-Kaggle Project 3: **

# Import Necessary Libraries
* Rather than not knowing which libraries have been imported or importing the same library multiple times, I moved all of this code up front.
* You also have to import enable_halving_search_cv from sklearn.experimental before you can import HalvingRandomSearchCV

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron


# Create the data frame and preprocess the training data

**Load Data** 
* Load train.csv into the dataframe, and take a look at the data to see how it looks.
* Take a look at the data to see what it looks like (head, shape, describe, info)
* See if there are any superfluous or unnecessary columns
* Check if there are any null or missing values.

In [2]:
train_data=pd.read_csv('/kaggle/input/forest-cover-type/train.csv')
train_data.head(10)

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,label,id
0,3031,38,8,192,40,30,220,223,138,1176,...,0,0,0,0,0,0,0,0,2,255258
1,2717,66,14,384,-19,927,233,211,108,2452,...,0,0,0,0,0,0,0,0,2,158435
2,3172,24,9,300,27,2389,215,220,141,2408,...,0,0,0,0,0,0,0,0,1,204891
3,3089,67,2,285,-37,2424,221,235,151,1745,...,0,0,0,0,0,0,0,0,2,365229
4,2569,22,20,216,87,1095,204,194,121,1506,...,0,0,0,0,0,0,0,0,2,562782
5,3192,65,11,792,111,1253,230,218,119,2356,...,0,0,0,0,0,0,0,0,1,357020
6,2892,216,14,228,96,1266,205,253,181,2751,...,0,0,0,0,0,0,0,0,2,314068
7,3133,175,8,134,11,3094,225,245,153,1289,...,0,0,0,0,0,0,0,0,2,480895
8,2911,45,10,272,-11,595,223,219,130,1950,...,0,0,0,0,0,0,0,0,1,499872
9,3013,159,10,42,0,2969,231,242,142,1537,...,1,0,0,0,0,0,0,0,1,429174


In [3]:
train_data.shape

(464809, 56)

In [4]:
#Check for any missing values
train_data.isnull().sum()

Elevation                             0
Aspect                                0
Slope                                 0
Horizontal_Distance_To_Hydrology      0
Vertical_Distance_To_Hydrology        0
Horizontal_Distance_To_Roadways       0
Hillshade_9am                         0
Hillshade_Noon                        0
Hillshade_3pm                         0
Horizontal_Distance_To_Fire_Points    0
Wilderness_Area1                      0
Wilderness_Area2                      0
Wilderness_Area3                      0
Wilderness_Area4                      0
Soil_Type1                            0
Soil_Type2                            0
Soil_Type3                            0
Soil_Type4                            0
Soil_Type5                            0
Soil_Type6                            0
Soil_Type7                            0
Soil_Type8                            0
Soil_Type9                            0
Soil_Type10                           0
Soil_Type11                           0


In [5]:
train_data.describe()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,label,id
count,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,...,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000,464809.000000
mean,2959.628663,155.778485,14.101696,269.369883,46.467265,2350.858288,212.141103,223.350170,142.564879,1983.065569,...,0.077761,0.002741,0.003259,0.000217,0.000521,0.026723,0.023797,0.015056,2.050812,290466.612862
std,279.823965,111.899874,7.489081,212.433304,58.309570,1557.941306,26.770894,19.754518,38.297524,1325.209370,...,0.267795,0.052282,0.056998,0.014739,0.022812,0.161272,0.152416,0.121774,1.396340,167808.139484
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
25%,2810.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1025.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,145042.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1999.000000,218.000000,226.000000,143.000000,1712.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,290336.000000
75%,3163.000000,261.000000,18.000000,384.000000,69.000000,3329.000000,231.000000,237.000000,168.000000,2551.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,435906.000000
max,3858.000000,360.000000,66.000000,1397.000000,601.000000,7112.000000,254.000000,254.000000,254.000000,7173.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000,581012.000000


In [6]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464809 entries, 0 to 464808
Data columns (total 56 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           464809 non-null  int64
 1   Aspect                              464809 non-null  int64
 2   Slope                               464809 non-null  int64
 3   Horizontal_Distance_To_Hydrology    464809 non-null  int64
 4   Vertical_Distance_To_Hydrology      464809 non-null  int64
 5   Horizontal_Distance_To_Roadways     464809 non-null  int64
 6   Hillshade_9am                       464809 non-null  int64
 7   Hillshade_Noon                      464809 non-null  int64
 8   Hillshade_3pm                       464809 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  464809 non-null  int64
 10  Wilderness_Area1                    464809 non-null  int64
 11  Wilderness_Area2                    464809 non-null 

**Pre-Process Data**
* Drop any unnecessary columns.
* It looks ike categorical data has already been converted to numerical values, so there is no need to encode any.
* There are no missing values, so there is no need to impute.
* Split data into features (X) and labels (y). I used 70% training and 30% testing.

In [7]:
#Drop "id" because it is not useful to train the model.
train_data=train_data.drop(['id'],axis=1)
#See what values are there for the label column
train_data['label'].value_counts()

label
2    226699
1    169565
3     28485
7     16424
6     13871
5      7564
4      2201
Name: count, dtype: int64

**Pre-Process Data**
* Split data into features (X) and labels (y) using train_test_split. I used 70% training and 30% testing but played around with an 80/20 split also.
* Set `random_state` to a number to get reproducible results.

In [8]:
# Split the dataset into training and testing sets
# Separating the features (X) and the target variable (y)
X = train_data.drop('label', axis=1).values
y = train_data['label'].values

#Use train_test_split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=17)

## Create a pipeline that sequentially applies a list of transformers and a final estimator

In [9]:
pipe_lr=make_pipeline(StandardScaler(),PCA(n_components=2),LogisticRegression())
pipe_lr.fit(X_train,y_train)
y_pred=pipe_lr.predict(X_test)
test_acc=pipe_lr.score(X_test,y_test)
print(f'Test accuracy: {test_acc:.3f}')

Test accuracy: 0.540


## K-Fold Cross-Validation

In [10]:
kfold = StratifiedKFold(n_splits=10).split(X_train, y_train)
scores = []
for k, (train, test) in enumerate(kfold):
    pipe_lr.fit(X_train[train], y_train[train])
    score = pipe_lr.score(X_train[test], y_train[test])
    scores.append(score)
    print(f'Fold: {k+1:02d}, '
    f'Class distr.: {np.bincount(y_train[train])}, '
    f'Acc.: {score:.3f}')
    
mean_acc = np.mean(scores)
std_acc = np.std(scores)
print(f'\nCV accuracy: {mean_acc:.3f} +/- {std_acc:.3f}')

Fold: 01, Class distr.: [     0 122012 163389  20498   1566   5436   9969  11792], Acc.: 0.542
Fold: 02, Class distr.: [     0 122012 163389  20498   1566   5436   9969  11792], Acc.: 0.540
Fold: 03, Class distr.: [     0 122012 163389  20498   1566   5436   9969  11792], Acc.: 0.541
Fold: 04, Class distr.: [     0 122013 163389  20497   1566   5436   9969  11792], Acc.: 0.536
Fold: 05, Class distr.: [     0 122012 163390  20497   1566   5436   9969  11792], Acc.: 0.536
Fold: 06, Class distr.: [     0 122012 163390  20497   1566   5436   9969  11792], Acc.: 0.537
Fold: 07, Class distr.: [     0 122012 163390  20497   1566   5436   9969  11792], Acc.: 0.540
Fold: 08, Class distr.: [     0 122012 163390  20497   1566   5436   9970  11792], Acc.: 0.539
Fold: 09, Class distr.: [     0 122012 163390  20498   1566   5436   9970  11791], Acc.: 0.540
Fold: 10, Class distr.: [     0 122012 163390  20498   1566   5436   9970  11791], Acc.: 0.536

CV accuracy: 0.539 +/- 0.002


In [11]:
scores = cross_val_score(estimator=pipe_lr,X=X_train,y=y_train,cv=10,n_jobs=1)
print(f'CV accuracy scores: {scores}')
print(f'CV accuracy: {np.mean(scores):.3f} '
    f'+/- {np.std(scores):.3f}')

CV accuracy scores: [0.54218099 0.54011026 0.54121285 0.5363453  0.53610327 0.53715208
 0.54008337 0.53918352 0.53950624 0.53595633]
CV accuracy: 0.539 +/- 0.002


## Grid Search

In [ ]:
pipe_svc = make_pipeline(StandardScaler(),
                         SVC(random_state=1))

param_range = [0.0001, 1.0, 10.0]

param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma': param_range, 
               'svc__kernel': ['rbf']}]

gs = GridSearchCV(estimator=pipe_svc, 
                  param_grid=param_grid, 
                  scoring='accuracy', 
                  refit=True,
                  cv=10)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)
clf = gs.best_estimator_
print(f'Test accuracy: {clf.score(X_test, y_test):.3f}')

## Randomized Search

In [ ]:
pipe_svc = make_pipeline(
    StandardScaler(),
    SVC(random_state=1))

param_range = [0.0001, 1.0, 10.0]

param_grid = [{'svc__C': param_range,
               'svc__kernel': ['linear']},
              {'svc__C': param_range,
               'svc__gamma': param_range,
               'svc__kernel': ['rbf']}]


rs = RandomizedSearchCV(estimator=pipe_svc,
                        param_distributions=param_grid,
                        scoring='accuracy',
                        refit=True,
                        n_iter=20,
                        cv=10,
                        random_state=1,
                        n_jobs=-1)

rs = rs.fit(X_train, y_train)
print(rs.best_score_)
print(rs.best_params_)


# Successive Halving

Successive halving is a more resource-efficient hyperparmeter search that makes finding suitable hyperparameter configurations more
efficient by successively throwing outunpromising hyperparameter configurations until only one configuration remains.

I chose this since grid search and randomized search were taking an extremely long time and using up a lot of resources.

I also had tried to adjust to reduce the amount of time it took to run, but ultimately I went with the Random Forest since it initially gave me the highest score. The code for the other models is still below.

### Successive Halving with SVC
Train the SVC regression model and different values for `C`.

I started off with param_range to include .0001, .001, .01, 0.1, 1.0, 10.0, and 100.0 but reduced this after the first run as it took too much time and focused on the ones around the best one found. I also reduced N-candidates for the same reason.

In [ ]:
pipe_svc = make_pipeline(StandardScaler(),
                         SVC(random_state=1))

param_range = [0.0001, 1.0, 10.0]

param_grid = [{'svc__C': param_range, 
               'svc__kernel': ['linear']},
              {'svc__C': param_range, 
               'svc__gamma':['scale','auto'], 
               'svc__kernel': ['rbf']}]

hs = HalvingRandomSearchCV(
    pipe_svc,
    param_distributions=param_grid,
    n_candidates=10,
    resource='n_samples',
    factor=3,
    random_state=1,
    n_jobs=-1)

hs = hs.fit(X_train, y_train)
print(hs.best_score_)
print(hs.best_params_)

In [ ]:
clfsvc = hs.best_estimator_
print(f'Test accuracy: {hs.score(X_test, y_test):.3f}')

### Successive Halving with Logistic Regression
Train the logistic regression model and test values for `solver`, `C`, and `penalty`

In [ ]:
pipe_lr = make_pipeline(StandardScaler(),
                        LogisticRegression(random_state=1))

param_range = [0.0001, 1.0, 10.0]

param_grid = [{'logisticregression__C': param_range,
               'logisticregression__penalty': ['l1', 'l2'],
               'logisticregression__solver': ['liblinear', 'saga']}]

lr = HalvingRandomSearchCV(
    pipe_lr,
    param_distributions=param_grid,
    n_candidates='exhaust',
    resource='n_samples',
    factor=3,
    random_state=1,
    n_jobs=-1)


In [ ]:
lr = lr.fit(X_train, y_train)
print(lr.best_score_)
print(lr.best_params_)

In [ ]:
clflr = lr.best_estimator_
print(f'Test accuracy: {hs.score(X_test, y_test):.3f}')

### Successive Halving with Perceptron
Use successive halving with the perceptron model using`eta0` as the learning rate to see which gives us the highest accuracy.

In [ ]:
pipe_p = make_pipeline(StandardScaler(),
                       Perceptron(random_state=1))

eta0_range = [0.0001, 0.01, 1.0]

param_grid = [{
    'perceptron__eta0': eta0_range,
    'perceptron__penalty': [None, 'l1', 'l2', 'elasticnet']
}]

percep = HalvingRandomSearchCV(
    pipe_p,
    param_distributions=param_grid,
    n_candidates='exhaust',
    resource='n_samples',
    factor=3,
    random_state=1)


In [ ]:
percep = percep.fit(X_train, y_train)
print(percep.best_score_)
print(percep.best_params_)

In [ ]:
clfp = percep.best_estimator_
print(f'Test accuracy: {hs.score(X_test, y_test):.3f}')

## Successive Halving with Random Forest


In [15]:
pipe_rf = make_pipeline(StandardScaler(),
                        RandomForestClassifier(random_state=1))

param_grid = [{
    'randomforestclassifier__n_estimators': [100, 200, 300, 400, 500],
    'randomforestclassifier__max_depth': [None, 10, 20, 30, 40, 50],
    'randomforestclassifier__min_samples_split': [2, 5, 10],
    'randomforestclassifier__min_samples_leaf': [1, 2, 4]
}]

rf = HalvingRandomSearchCV(
    pipe_rf,
    param_distributions=param_grid,
    n_candidates='exhaust',
    resource='n_samples',
    factor=2,
    random_state=1,
    n_jobs=-1)


In [16]:
rf = rf.fit(X_train, y_train)
print(rf.best_score_)
print(rf.best_params_)

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:305: UserWarning: The total space of parameters 270 is smaller than n_iter=5312. Running 270 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


0.8313335418244886
{'randomforestclassifier__n_estimators': 300, 'randomforestclassifier__min_samples_split': 2, 'randomforestclassifier__min_samples_leaf': 1, 'randomforestclassifier__max_depth': 50}


In [17]:
clfrf = rf.best_estimator_
print(f'Test accuracy: {rf.score(X_test, y_test):.3f}')

Test accuracy: 0.951


# Load the test data and apply the best model to the test data

In [ ]:
#Load the test data and apply the model to the test data
test_data = pd.read_csv('/kaggle/input/forest-cover-type/test.csv')

#drop the 'id' variable we dropped in the training data
test_data1 = test_data.drop(['id'], axis=1)


# Predict income with the logistic regression model
test_pred = rf.predict(test_data1)

#Create the csv file for submission.

submission_file=test_data[['id']].copy()
submission_file['label'] = test_pred
submission_file.to_csv('Kaggle3submission.csv', index=False)
